In [197]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Elegimos el año

In [198]:
year = 2023

Cargamos los datos

In [199]:
data  = pd.read_csv(f'datos\counted\d{year}_counted.csv')

In [200]:
data = data.drop('Unnamed: 0', axis=1)

## TF-IDF
Requires text preprocesssing (e.g., lowercasing, removing punctuation, stop words, stemming)\
The algortihm will be used just over 'avisocuerpo' because 'avisocargo' and 'avisorequisitos' are very similar and the algorithm identifies everything giving nothing interessting for us

In [201]:
data['avisocuerpo_t'] = data['avisocuerpo'].apply(lambda x: str(x).lower() if not pd.isna(x) else '')
data['avisocuerpo_t'] = data['avisocuerpo_t'].apply(lambda x: x.replace('.', '').replace(',', '').replace(';', '').replace('-', '').replace('>', '').replace('<', '').replace('\r', '').
                                                replace('\n', '').replace('\n2', '').replace('\n1', '').replace('\n3', '').replace('\\', '').replace('/', '').replace('html', ''))
data['avisocuerpo_t'] = data['avisocuerpo_t'].fillna('vacio')

Model

Due to the size of the data is neccesary to do it by chunks

In [202]:
# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Define the chunk size and calculate the number of chunks
chunk_size = 1000  # You can adjust this as needed
num_chunks = len(data) // chunk_size + 1

# Initialize the 'duplicates' dictionary to store results
duplicates = {}

# Loop through the data in chunks
for chunk_num in range(num_chunks):
    start_idx = chunk_num * chunk_size
    end_idx = start_idx + chunk_size
    chunk_data = data.iloc[start_idx:end_idx].copy()  # Get a chunk of data
    
    # Fit and transform the job descriptions to TF-IDF vectors for the current chunk
    tfidf_matrix = tfidf_vectorizer.fit_transform(chunk_data['avisocuerpo_t'].fillna('Vacio'))

    # Calculate cosine similarity between job descriptions in the current chunk
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Define a similarity threshold, 0.5 to try to capture more similarities between the ads
    similarity_threshold = 0.50

    # Identify duplicates or similar job ads within the current chunk
    for i in range(len(chunk_data)):
        duplicates[start_idx + i] = [j for j, score in enumerate(cosine_sim[i]) if score > similarity_threshold and i != j]

# Now, the 'duplicates' dictionary contains the results for the entire dataset.

In [203]:
data['similars'] = duplicates

In [204]:
data.to_csv(f'datos\counted\d{year}_counted.csv')

In [205]:
year

2023